<img src='https://next.ific.uv.es/next/templates/rt_quasar_j15/images/logo/stylenext/logo.png' style='height: 40px' />

# Simulation of a fiber barrel for NEXT-100

## Kr event

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import ipywidgets as widgets

In [2]:
mpl.rcParams.update({
    "font.family": "serif",
    "axes.titlesize": "xx-large",
    "axes.labelsize": "xx-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
    "legend.fontsize": "large",
    "figure.titlesize": "xx-large",
    "legend.frameon": False,
    "text.usetex": True
})

In [3]:
kr_fibers = pd.read_hdf("fiber_kr.h5", "MC/sns_response")
kr_sensors = pd.read_hdf("fiber_kr.h5", "MC/sns_positions")
n_sipms = len(kr_sensors['sensor_id'])

In [4]:
event_id = 0
wfs_event = kr_fibers.query(f"event_id=={event_id}")

def gui(temperature, bin_width, dark_rate):
    temperature += 273.15
    fig, ax = plt.subplots(1, 1)
    x_range = (-2000 / 1e3, 4000 / 1e3)  # us
    time_window = x_range[1] - x_range[0]
    n_bins = int(time_window / bin_width)
    dark_rate *= 1e3 / 1e6  # counts per us
    temperature_reduction = pow(2, (298 - temperature) / 5.3)
    dark_pulses = np.random.uniform(
        x_range[0],
        x_range[1],
        int(dark_rate * time_window * n_sipms / temperature_reduction),
    )

    kr_times = wfs_event["time_bin"] / 1e3 / 10  # us

    n, bins, patches = ax.hist(
        kr_times,
        range=x_range,
        bins=n_bins,
        alpha=0.4,
        color="r",
        lw=2,
        label=r"$^{83m}$Kr",
        ls="--",
    )

    n_dark, bins, patches = ax.hist(
        dark_pulses,
        range=x_range,
        bins=n_bins,
        color="k",
        lw=2,
        ls="--",
        label=r"Dark pulses",
        alpha=0.4,
    )
    n, bins, patches = ax.hist(
        kr_times,
        range=x_range,
        bins=n_bins,
        bottom=n_dark,
        histtype="step",
        color="k",
        lw=2,
    )
    ax.set_xlim(x_range[0], x_range[1])
    ax.set_xlabel(r"Time [$\mathrm{\mu}s$]")
    ax.set_ylabel("Charge [p.e.]")
    ax.set_ylim(0, max(n + n_dark) * 1.3)
    ax.set_title(r"$^{83m}$Kr S1 signal at T=%i K" % temperature)
    ax.legend(fontsize='x-large')

In [5]:
style = {"description_width": "initial", "font-size": "x-large"}
layout = widgets.Layout(width="500px", fontsize="x-large")

temperature = widgets.IntSlider(
    description="Temperature [$^{\circ}$C]",
    style=style,
    layout=layout,
    min=-40,
    max=25,
    value=25,
)
bin_width = widgets.FloatSlider(
    description=r"Time sampling [$\mu$s]",
    style=style,
    layout=layout,
    min=0.1,
    max=2,
    value=1,
)

dark_rate = widgets.IntSlider(
    description="Dark rate per sensor [kcps]",
    style=style,
    layout=layout,
    min=1000,
    max=6000,
    value=2000,
    step=100
)

out = widgets.interactive_output(
    gui,
    {
        "temperature": temperature,
        "bin_width": bin_width,
        "dark_rate": dark_rate,
    },
)

ui = widgets.VBox(
    [
        temperature,
        bin_width,
        dark_rate
    ]
)
display(ui, out)

Output()